In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
# === 1. Chargement des données ===
df = pd.read_csv('dump_kadea.csv', header=None, on_bad_lines='skip')

In [3]:
# === 2. Attribution des noms de colonnes ===
df.columns = [
    'id_transaction',
    'date_vente',
    'code_magasin',
    'nom_magasin',
    'categorie',
    'nom_produit',
    'mode_paiement',
    'quantite',
    'prix_unitaire',
    'cout_unitaire',
    'reduction',
    'type_client',
    'code_vendeur'
]

# === 3. Nettoyage des dates ===
def parse_date_flexible(date_str):
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str).strip()
    # Supprimer les espaces et caractères parasites
    date_str = re.sub(r'\s+', ' ', date_str)
    formats = [
        '%Y-%m-%d',
        '%d/%m/%Y',
        '%d %B %Y',
        '%d %b %Y',
        '%B %d, %Y',
        '%d %B, %Y'
    ]
    for fmt in formats:
        try:
            return pd.to_datetime(datetime.strptime(date_str, fmt))
        except ValueError:
            continue
    return pd.NaT

df['date_vente'] = df['date_vente'].apply(parse_date_flexible)

# === 4. Suppression lignes critiques manquantes ===
required_cols = ['id_transaction', 'date_vente', 'code_magasin', 'categorie', 'quantite', 'prix_unitaire', 'cout_unitaire']
df_clean = df.dropna(subset=required_cols).copy()

# === 5. Conversion numérique sécurisée ===
df_clean['quantite'] = pd.to_numeric(df_clean['quantite'], errors='coerce')
df_clean['prix_unitaire'] = pd.to_numeric(df_clean['prix_unitaire'], errors='coerce')
df_clean['cout_unitaire'] = pd.to_numeric(df_clean['cout_unitaire'], errors='coerce')

# Suppression des lignes avec quantité/prix/coût non valides
df_clean = df_clean[
    (df_clean['quantite'] > 0) &
    (df_clean['prix_unitaire'] >= 0) &
    (df_clean['cout_unitaire'] >= 0)
].copy()

# === 6. Standardisation du mode de paiement ===
def normalize_paiement(mode):
    if pd.isna(mode):
        return 'Autre'
    mode = str(mode).strip().lower()
    if 'cash' in mode:
        return 'Cash'
    elif 'momo' in mode or 'mobile' in mode:
        return 'Mobile Money'
    elif 'carte' in mode or 'card' in mode:
        return 'Carte'
    else:
        return 'Autre'

df_clean['mode_paiement'] = df_clean['mode_paiement'].apply(normalize_paiement)

# === 7. Calcul des indicateurs métiers ===
df_clean['ca'] = df_clean['prix_unitaire'] * df_clean['quantite']
df_clean['marge_brute'] = (df_clean['prix_unitaire'] - df_clean['cout_unitaire']) * df_clean['quantite']

# === 8. Tri final ===
df_clean = df_clean.sort_values('date_vente').reset_index(drop=True)

# === 9. Export ===
df_clean.to_csv('kawa_ventes_nettoye.csv', index=False, encoding='utf-8')
print("✅ Nettoyage terminé. Fichier sauvegardé : kawa_ventes_nettoye.csv")

✅ Nettoyage terminé. Fichier sauvegardé : kawa_ventes_nettoye.csv
